In [3]:
#Here are a set of libraries we imported to complete this assignment. 
#Feel free to use these or equivalent libraries for your implementation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt # this is used for the plot the graph 
import os
import seaborn as sns # used for plot interactive graph.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC  
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample # module for resampling
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

import statistics
import math
import time
from datetime import datetime

from matplotlib import pyplot
import itertools

%matplotlib inline

import random 
  
random.seed(42) 

In [4]:
#read in the csv
MY_PATH = "./datasets/training_data_cleaned.csv"
data = pd.read_csv(MY_PATH)

# Take a look at the data

In [5]:
data.head()

,Unnamed: 0,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,...,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,date,recent_date
0,0,2021-01-20,3,2021,30222,U0117613,Owner Operator,Rosas Transport,Santa Fe Springs,CA,...,4,1,253,254,0,3,3,257,2021-01-20,2021-02-17
1,1,2020-03-09,1,2020,4524,C0090405,Fleet,Daylight Transport,Long Beach,CA,...,6,0,0,0,2671,0,2671,2671,2020-03-09,2021-02-15
2,2,2020-02-14,5,2020,31164,C0097099,Fleet,First Class Transport #8,Hawthorne,CA,...,1,29,82,111,30,19,49,160,2020-02-14,2020-12-18
3,3,2019-06-10,1,2019,8859,U0097935,Owner Operator,DTELLIS TRUCKING,Long Beach,CA,...,1,25,6,31,0,1,1,32,2019-06-10,2020-02-16
4,4,2019-12-03,2,2019,18870,U0106560,Owner Operator,Z&N Transport,Los Angeles,CA,...,4,1,233,234,0,9,9,243,2019-12-03,2020-02-18


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                83414 non-null  int64  
 1   dt                        83414 non-null  object 
 2   weekday                   83414 non-null  int64  
 3   year                      83414 non-null  int64  
 4   id_driver                 83414 non-null  int64  
 5   id_carrier_number         83414 non-null  object 
 6   dim_carrier_type          83414 non-null  object 
 7   dim_carrier_company_name  83367 non-null  object 
 8   home_base_city            83369 non-null  object 
 9   home_base_state           83369 non-null  object 
 10  carrier_trucks            83414 non-null  object 
 11  num_trucks                83344 non-null  float64
 12  interested_in_drayage     83414 non-null  object 
 13  port_qualified            83414 non-null  object 
 14  signup

In [7]:
data.describe()

,Unnamed: 0,weekday,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
count,83414.000000,83414.000000,83414.000000,83414.000000,83344.000000,71142.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000
mean,41706.500000,3.500060,2018.960870,18224.058060,22.579538,298.646201,2.075659,29.536241,71.511533,101.047774,148.027561,13.080166,161.107728,266.273371
std,24079.692014,1.687443,1.359275,11667.851627,48.816676,390.377138,2.669705,88.308416,194.548817,214.587614,415.376825,42.274194,413.197837,448.353468
min,0.000000,1.000000,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,20853.250000,2.000000,2018.000000,7890.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000
50%,41706.500000,3.000000,2019.000000,16299.000000,4.000000,61.000000,1.000000,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000
75%,62559.750000,5.000000,2020.000000,28974.000000,14.000000,497.000000,2.000000,23.000000,18.000000,94.000000,110.000000,1.000000,135.000000,325.000000
max,83413.000000,7.000000,2021.000000,38096.000000,195.000000,1653.000000,129.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000


In [8]:
# Drop extra column

data = data.drop(labels=['Unnamed: 0'], axis=1)
data.head()

,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,date,recent_date
0,2021-01-20,3,2021,30222,U0117613,Owner Operator,Rosas Transport,Santa Fe Springs,CA,"[""poweronly""]",...,4,1,253,254,0,3,3,257,2021-01-20,2021-02-17
1,2020-03-09,1,2020,4524,C0090405,Fleet,Daylight Transport,Long Beach,CA,"[""dryvan""]",...,6,0,0,0,2671,0,2671,2671,2020-03-09,2021-02-15
2,2020-02-14,5,2020,31164,C0097099,Fleet,First Class Transport #8,Hawthorne,CA,"[""poweronly"", ""dryvan""]",...,1,29,82,111,30,19,49,160,2020-02-14,2020-12-18
3,2019-06-10,1,2019,8859,U0097935,Owner Operator,DTELLIS TRUCKING,Long Beach,CA,"[""dryvan""]",...,1,25,6,31,0,1,1,32,2019-06-10,2020-02-16
4,2019-12-03,2,2019,18870,U0106560,Owner Operator,Z&N Transport,Los Angeles,CA,"[""dryvan""]",...,4,1,233,234,0,9,9,243,2019-12-03,2020-02-18


# Part 1: Generate labels
## drivers in the 75th percentile of ‘loads’ and the 75th percentile of ‘most_recent_load_date’ are assigned a label of 1 (indicating a high performing driver) with all others being assigned a 0 - (NOTE: your labels will likely be unbalanced. 

### pre-work from Piazza that must be done

- collapse your rows by trimming any duplicative entries per-driver, and only leaving the most recent (and therefore cumulative with regards to features like loads) entry. 
- Thereby each row is it's own driver, and it more closely fits the desired model. 
- If you take this approach, it would be worth playing around with different augmentations to provide additional nuance to your data. (e.g., while you have a total count of loads, you don't have a clear indication of the distribution of those loads. Is this an older driver who was active several years ago but now works irregularly, or is this a new driver that's been very active in the last few months?) Boiling up that kind of data into your features could be incredibly powerful.

In [31]:
def preprocess_data(dataset, testing=False):

    #for each duplicate of a driver id 'id_driver', cumulate the 'load' values for this driver 
    #AND add the median load for this driver as a new column 'median_load'

    drivers = list(set(dataset['id_driver'])) #get rid of duplicates
    rows_to_keep = []
    median_loads = []

    for d in drivers:
        loads_for_driver = dataset.loc[dataset['id_driver'] == d]
        
        most_recent_date = ""
        
        if testing:
            dates = list(data.loc[data['id_driver'] == d]['dt'])
            dates.sort(reverse=True)
            most_recent_date = dates[0]
        else:
            most_recent_date = list(loads_for_driver['most_recent_load_date'])[0]

        loads = sum(loads_for_driver['loads'])
        median_load = statistics.median(list(loads_for_driver['loads']))

        row = loads_for_driver.loc[loads_for_driver['dt'] == most_recent_date]
        if row.empty:
            dates = list(data.loc[data['id_driver'] == d]['dt'])
            dates.sort(reverse=True)
            next_recent_date = dates[0]
            row = loads_for_driver.loc[loads_for_driver['dt'] == next_recent_date]
        
        row['median_load'] = median_load
        row['loads'] = loads
        
        if len(row) > 1:
            row = row.head(1)

        rows_to_keep.append(row)
        
    #create new dataframe with unique drivers
    unique_data = pd.concat(rows_to_keep)
    print("rows to keep:", len(rows_to_keep))
    print("rows in unique_data:", len(unique_data))
    
    return unique_data


In [32]:
# DF with one row per driver, created from rows_to_keep -- ISSUE somehow goes from 5288 -> 5255

condensed_data = preprocess_data(data)
condensed_data.head()

# condensed_data = pd.concat(rows_to_keep)
# print("rows to keep:", len(rows_to_keep))
# print("rows in condensed_data:", len(condensed_data))
# condensed_data.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


rows to keep: 5288
rows in unique_data: 5288


,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,date,recent_date,median_load
12922,2016-06-20,1,2016,20,U0000035,Owner Operator,Mega Fleet Corp,Pasadena,CA,"[""dryvan""]",...,0,0,0,42,0,42,42,2016-06-20,2016-06-20,1.0
17520,2015-10-29,4,2015,26,U0000041,Owner Operator,NaN,"Sierra Madre,",CA,"[""dryvan""]",...,0,0,0,1,0,1,1,2015-10-29,2015-10-29,1.0
30284,2015-12-09,3,2015,27,U0000042,Owner Operator,NaN,"Sierra Madre,",CA,"[""dryvan""]",...,0,0,0,11,0,11,11,2015-12-09,2015-12-09,1.0
61261,2018-12-05,3,2018,30,U0000045,Owner Operator,A & J Transport,Riverside,CA,"[""reefer""]",...,0,0,0,4,0,4,4,2018-12-05,2018-12-05,1.0
25636,2016-04-01,5,2016,31,U0000046,Owner Operator,lv transportation llc,NaN,NaN,"[""dryvan""]",...,0,0,0,15,0,15,15,2016-04-01,2016-04-01,1.0


In [11]:
condensed_data.describe()

,weekday,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,median_load
count,5291.000000,5291.000000,5291.000000,5249.000000,3963.000000,5291.000000,5291.000000,5291.000000,5291.000000,5291.000000,5291.000000,5291.000000,5291.000000,5291.000000
mean,3.329049,2018.713098,17732.668494,16.719947,290.242745,32.814591,4.724816,9.245700,13.970516,15.136458,2.383481,17.519940,33.113022,1.258836
std,1.638163,1.434551,11624.261142,40.745352,368.000082,123.107403,20.706297,59.876237,64.959332,99.420194,17.742087,100.784399,124.357840,1.425714
min,1.000000,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2.000000,2018.000000,8049.500000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,3.000000,2019.000000,15228.000000,4.000000,108.000000,3.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,3.000000,1.000000
75%,5.000000,2020.000000,29290.500000,11.000000,488.500000,17.000000,2.000000,0.000000,4.000000,4.000000,0.000000,5.000000,17.000000,1.000000
max,7.000000,2021.000000,38096.000000,195.000000,1653.000000,4250.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000,80.000000


In [12]:
condensed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5291 entries, 12922 to 44688
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dt                        5291 non-null   object 
 1   weekday                   5291 non-null   int64  
 2   year                      5291 non-null   int64  
 3   id_driver                 5291 non-null   int64  
 4   id_carrier_number         5291 non-null   object 
 5   dim_carrier_type          5291 non-null   object 
 6   dim_carrier_company_name  5284 non-null   object 
 7   home_base_city            5279 non-null   object 
 8   home_base_state           5279 non-null   object 
 9   carrier_trucks            5291 non-null   object 
 10  num_trucks                5249 non-null   float64
 11  interested_in_drayage     5291 non-null   object 
 12  port_qualified            5291 non-null   object 
 13  signup_source             5291 non-null   object 
 14  ts_

In [ ]:
# compute the 75th percentile for total loads 
# (piazza @222 says to use 75th percentile from whole dataset before dropping duplicates)
load_percentile = data["total_loads"].quantile(0.75)
print("75th percentile of total_loads for full dataset:", load_percentile)

In [ ]:
# convert date to date object 
data.most_recent_load_date = [pd.to_datetime(d) for d in data.most_recent_load_date]
condensed_data.most_recent_load_date = [pd.to_datetime(d) for d in condensed_data.most_recent_load_date]

# calculate 75th percentile for most_recent_load_date in full dataset 
date_percentile = data["most_recent_load_date"].quantile(0.75)
print("75th percentile of most_recent_load_date for full dataset:", date_percentile)

In [ ]:
# create a new column and use np.where to assign values based on most_recent_load_date and total_loads
condensed_data["high_performing"] = np.where((condensed_data["most_recent_load_date"] >= date_percentile) 
                                             & (condensed_data["total_loads"] >= load_percentile), 1, 0)

n_high_performers = len(condensed_data.loc[condensed_data["high_performing"] == 1])
n_non_high_performers = len(condensed_data.loc[condensed_data["high_performing"] == 0])


print("high performing drivers:", n_high_performers)
print("low performing drivers:", n_non_high_performers)

## Balance data

In [ ]:
# balancing data by upsampling the high-performing drivers
majority = condensed_data[condensed_data["high_performing"] == 0]
minority = condensed_data[condensed_data["high_performing"] == 1]

# want to sample high-performers with replacement to match the number of non-high performing drivers
minority_upsampled = resample(minority, replace=True, n_samples=n_non_high_performers, random_state=42)
upsampled_data = pd.concat([majority, minority_upsampled])
upsampled_data["high_performing"].value_counts()

# Step 2: Drop ‘load’ and ‘most_recent_load_date’ from your data frame 
Since those fields are
being directly used to label your data please remove them from your training and
testing cohorts.

In [ ]:
# condensed_data = condensed_data.drop(labels=['total_loads', 'most_recent_load_date'], axis=1)
# condensed_data.info()

upsampled_data

upsampled_data = upsampled_data.drop(labels=['total_loads', 'most_recent_load_date'], axis=1)
upsampled_data.info()

# 3. Run some basic statistics on your variables including correlations with labels and report findings 

In [ ]:
print(upsampled_data['high_performing'].value_counts())
upsampled_data['high_performing'].hist()
plt.show()

In [ ]:
upsampled_data.describe()

In [ ]:
upsampled_data.hist(figsize=(20,15))
plt.show()

In [ ]:
corr_matrix = upsampled_data.corr()
corr_matrix

In [ ]:
corr_matrix["high_performing"].sort_values(ascending=False)

In [ ]:
attributes = ["high_performing", "marketplace_loads", "marketplace_loads_atlas", "median_load",
              "brokerage_loads_atlas", 'brokerage_loads', 'brokerage_loads_otr']
scatter_matrix(upsampled_data[attributes], figsize=(20, 15))
plt.show()

### Findings: 

All of the loads-related columns are extremely right-skewed. On average, drivers have been assigned more jobs through brokers than through the NEXT marketplace. For loads assigned through brokers, they are more likely to be OTR loads than drayage jobs. The median loads that a driver carries per job is 1.

`marketplace_loads` and `marketplace_loads_atlas` have a very high correlation with `high_performing` of over 0.5 each. `brokerage_loads_atlas`, `brokerage_loads`, and `brokerage_loads_otr` also have moderate correlations with `high_performing`. `marketplace_loads` is highly correlated to `marketplace_loads_atlas` and moderately correlated to `marketplace_loads_otr`, showing some collinearity within the dataset. `brokerage_loads` is very highly correlated to `brokerage_loads_otr`. 

# 4. Create a data feature extraction plan and implement a pipeline to execute it

a. Determine which fields to retain and which to drop.

b. For those you retain, determine a categorization strategy

c. Determine an imputation strategy (you should choose more than one imputation
method depending on the specifics of your data

d. Augment at least one feature, ideally a feature cross, or non-linear transition

e. Determine a strategy for scaling features

f. Implement a single pipeline to execute this transformation

g. Document your data strategy in your report. Provide an explanation or
justification for why you chose the data you did, and also detail any experiments
you ran and the results

In [ ]:
upsampled_data.isnull().sum()

## Pipeline

In [ ]:
unlabeled = upsampled_data.drop("high_performing", axis=1)
labels = upsampled_data["high_performing"].copy()

#### pipeline function

In [ ]:
def pipeline(unlabeled):

    # drop features 
    unlabeled = unlabeled.drop(labels=['load_day',                       #redundant on dt
                                      'id_driver','id_carrier_number',    #not relevant
                                      'dim_carrier_company_name', 'home_base_city', #too many values to OHE
                                      'dim_preferred_lanes', 'ts_first_approved', 'days_signup_to_approval',  #too many null values
                                      ], axis=1) 

    #impuatation
    state_mode = str(unlabeled['home_base_state'].mode())
    unlabeled['home_base_state'] = unlabeled['home_base_state'].fillna(state_mode) # mode value - 87% are CA 

    truck_median = unlabeled['num_trucks'].median()
    unlabeled['num_trucks'] = unlabeled['num_trucks'].fillna(truck_median) # right-skewed so use median
    
    #augmentation
    unlabeled["marketplace_ratio"] = unlabeled["marketplace_loads"]/(unlabeled["marketplace_loads"]+unlabeled["brokerage_loads"])
    
    #categorization
    
    # change date representations to ints
    unlabeled['dt'] = unlabeled['dt'].apply(lambda d : int(pd.to_datetime(d).timestamp()))
    unlabeled['ts_signup'] = unlabeled['ts_signup'].apply(lambda d : int(pd.to_datetime(d).timestamp()))
    unlabeled['first_load_date'] = unlabeled['first_load_date'].apply(lambda d : int(pd.to_datetime(d).timestamp()))
    unlabeled['recent_date'] = unlabeled['recent_date'].apply(lambda d : int(pd.to_datetime(d).timestamp()))


    # group by numeric or categorical
    data_num = ['year','num_trucks', 'loads', 
                'marketplace_loads_otr', 'marketplace_loads_atlas','marketplace_loads', 
                'brokerage_loads_otr', 'brokerage_loads_atlas', 'brokerage_loads', 
                'median_load', 'dt', 'ts_signup','first_load_date', 'recent_date']
    data_cat = ['weekday', 'dim_carrier_type','home_base_state', 'carrier_trucks', 'interested_in_drayage', 'port_qualified',
               'signup_source', 'driver_with_twic']
    
    #full pipeline
    num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
    #         ('attribs_adder', AugmentFeatures()),
            ('std_scaler', StandardScaler()),
        ])

    numerical_features = data_num
    categorical_features = data_cat

    full_pipeline = ColumnTransformer([
            ("num", num_pipeline, numerical_features),
            ("cat", OneHotEncoder(), categorical_features),
        ])

    prepared_data = full_pipeline.fit_transform(unlabeled)
    return prepared_data

    

In [ ]:
prepared_data = pipeline(unlabeled)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(prepared_data, labels, test_size=0.2, random_state=10)

# 5. Implement a basic Logistic Regression - 
** piazza post corrected to logistic from linear

With your newly pipelined data, find and interpret important features (e.g. using regression and associated p-values). If there are any collinearities be careful when incorporating them into the regression.

In [ ]:
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)
log_predicted = log_reg.predict(X_test)

print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(y_test, log_predicted)))
print("%-12s %f" % ('Precision:', metrics.precision_score(y_test, log_predicted)))
print("%-12s %f" % ('Recall:', metrics.recall_score(y_test, log_predicted)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(y_test, log_predicted)))

#ROC curve
metrics.plot_roc_curve(log_reg, X_test, y_test)  
plt.show()  

In [ ]:
log_reg.coef_.shape

### not sure how to interpret important features or p-values?? watching this piazza post https://piazza.com/class/kjdcwsz1c306g7?cid=251

# 6. PCA

We used TruncatedSVD (singular value decomposition) instead of PCA for this step because SVD works better with sparse matrices.

In [ ]:
svd = TruncatedSVD(n_components=10, random_state=10)
svd.fit(X_train)
print(svd.explained_variance_ratio_)

X_train_transformed = svd.transform(X_train)
X_test_transformed = svd.transform(X_test)

# 7. Ensemble Method

In [ ]:
# Employ an ensemble method

bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=0)
bc.fit(X_train_transformed, y_train)
bc_pred = bc.predict(X_test_transformed)


print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(y_test, bc_pred)))
print("%-12s %f" % ('Precision:', metrics.precision_score(y_test, bc_pred)))
print("%-12s %f" % ('Recall:', metrics.recall_score(y_test, bc_pred)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(y_test, bc_pred)))
# BRUH

# 8. Neural Network

In [ ]:
nn = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(50,50))
nn.fit(X_train_transformed, y_train)
nn_pred = nn.predict(X_test_transformed)

print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(y_test, nn_pred)))
print("%-12s %f" % ('Precision:', metrics.precision_score(y_test, nn_pred)))
print("%-12s %f" % ('Recall:', metrics.recall_score(y_test, nn_pred)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(y_test, nn_pred)))

# 9. Cross Validation

Cross-Validate your training results - Employ K-Fold Cross-validation to your training
regimen for both ensemble and NN classifers. (Optional: employ a stratifiedshufflesplit
as well to ensure equitable distribution along a key parameter)

In [ ]:
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std

transformed_data = svd.transform(prepared_data)

# Ensemble
cv = KFold(n_splits=10)
bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=0)
# create model
scores = cross_val_score(bc, transformed_data, labels, scoring='accuracy', cv=cv)

# report performance
print('Accuracy: %.4f (%.3f)' % (mean(scores), std(scores)))


# NN
cv = KFold(n_splits=10)
nn = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(50,50))

# evaluate model
scores = cross_val_score(nn, transformed_data, labels, scoring='accuracy', cv=cv)
# report performance
print('Accuracy: %.4f (%.3f)' % (mean(scores), std(scores)))

# 10. Custom Models

### Support Vector Machine

In [ ]:
svm = SVC(random_state=10) 
svm.fit(X_train_transformed, y_train)

svm_pred = svm.predict(X_test_transformed) 

print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(y_test, svm_pred))) 
print("%-12s %f" % ('Precision:', metrics.precision_score(y_test, svm_pred)))
print("%-12s %f" % ('Recall:', metrics.recall_score(y_test, svm_pred)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(y_test, svm_pred)))

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_transformed, y_train)
gnb_pred = gnb.predict(X_test_transformed)

print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(y_test, gnb_pred))) 
print("%-12s %f" % ('Precision:', metrics.precision_score(y_test, gnb_pred)))
print("%-12s %f" % ('Recall:', metrics.recall_score(y_test, gnb_pred)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(y_test, gnb_pred)))